In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
OHE = OneHotEncoder(sparse=False)
pd.set_option('display.max_rows', 30)

In [2]:
# 月日如果用起来要one-hot，之后产生的feature就太多了而且感觉没必要，可能考虑is_special更合适
# 但是前后依赖这里怎么办呢？
# train[['month','day']] = train.date.str.split('-', expand=True)
def get_x_y(df):
    data = df.drop(columns=['Unnamed: 0', 'route', 'calculated_day', 'date'])
    ohe = OHE.fit_transform(data[['hr', 'weekday']])
    ohe_df = pd.DataFrame(ohe, columns= OHE.get_feature_names(input_features=['hr', 'weekday']))
    data = pd.concat([data, ohe_df],axis = 1)
    data = data.drop(columns=['hr', 'weekday'])
    x = data.drop(columns=['label'])
    y = data.label
    return x, y

In [3]:
def mape_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))


def scoring(reg, x, y):
    pred = reg.predict(x)
    return -mape_error(pred, y)

## Read the Real Value at Each Time Window

In [10]:


import math
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
names=locals()

link_time={}
time_train={}
time_predict={}
time_check={}
test_time_train={}
test_time_check={}
weathers={}
rainingTotalTime={}

file_path='/Users/vayne/Desktop/DM_Project_0522due/dataSet_phase2/table5.csv'
# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '_' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        travel_seq = each_traj[4]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = math.floor(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        time=start_time_window
        tt = float(each_traj[-1])
        # 國慶不管，因道路狀態不一樣
        if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
            continue
        # 中秋節不管，因道路狀態不一樣
        if(time.month==9 and time.day in [15,16,17]):
            continue
        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

In [11]:
real_value=[]
real_value_dict={}
#fw = open('/Users/vayne/Desktop/dm_pro_engin/real_value.csv', 'w')
#fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
for route in travel_times.keys():
    if route not in real_value_dict.keys():
        real_value_dict[route] = {}
    route_time_windows = list(travel_times[route].keys())
    route_time_windows.sort()
    for time_window_start in route_time_windows:
        time_window_end = time_window_start + timedelta(minutes=20)
        tt_set = travel_times[route][time_window_start]
        avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
        out_line =[str(route),time_window_start, str(avg_tt)]
        real_value.append(out_line)    
        
        

In [12]:
real_value

[['B_3', datetime.datetime(2016, 10, 18, 0, 0), '64.69'],
 ['B_3', datetime.datetime(2016, 10, 18, 0, 40), '75.37'],
 ['B_3', datetime.datetime(2016, 10, 18, 1, 40), '70.14'],
 ['B_3', datetime.datetime(2016, 10, 18, 2, 0), '96.03'],
 ['B_3', datetime.datetime(2016, 10, 18, 3, 40), '76.88'],
 ['B_3', datetime.datetime(2016, 10, 18, 6, 0), '54.75'],
 ['B_3', datetime.datetime(2016, 10, 18, 6, 20), '121.02'],
 ['B_3', datetime.datetime(2016, 10, 18, 6, 40), '94.04'],
 ['B_3', datetime.datetime(2016, 10, 18, 7, 0), '120.6'],
 ['B_3', datetime.datetime(2016, 10, 18, 7, 20), '73.4'],
 ['B_3', datetime.datetime(2016, 10, 18, 7, 40), '89.91'],
 ['B_3', datetime.datetime(2016, 10, 18, 8, 0), '126.4'],
 ['B_3', datetime.datetime(2016, 10, 18, 8, 20), '115.52'],
 ['B_3', datetime.datetime(2016, 10, 18, 8, 40), '98.86'],
 ['B_3', datetime.datetime(2016, 10, 18, 9, 0), '131.15'],
 ['B_3', datetime.datetime(2016, 10, 18, 9, 20), '151.95'],
 ['B_3', datetime.datetime(2016, 10, 18, 9, 40), '117.69'],

In [13]:
real_value_needed=[]
for i in real_value:
    if i[1].hour in [8,9,17,18]:
        real_value_needed.append(i)
        

In [14]:
#convert to dataframe
feature=['route','time','label']
real_value_df=pd.DataFrame(real_value_needed,columns=feature)


In [15]:
real_value_df

,route,time,label
0,B_3,2016-10-18 08:00:00,126.4
1,B_3,2016-10-18 08:20:00,115.52
2,B_3,2016-10-18 08:40:00,98.86
3,B_3,2016-10-18 09:00:00,131.15
4,B_3,2016-10-18 09:20:00,151.95
...,...,...,...
456,B_1,2016-10-24 17:00:00,114.14
457,B_1,2016-10-24 17:20:00,128.66
458,B_1,2016-10-24 17:40:00,123.62
459,B_1,2016-10-24 18:00:00,146.24


In [ ]:
real_value_df_A_2= real_value_df.loc[real_value_df['route'] == 'A_2']
real_value_df_A_2.to_csv('real_value_df_A_2.csv', index=False)

In [128]:
real_value_df_A_3= real_value_df.loc[real_value_df['route'] == 'A_3']
real_value_df_A_3.to_csv('real_value_df_A_3.csv', index=False)

In [151]:
real_value_df_B_1= real_value_df.loc[real_value_df['route'] == 'B_1']
real_value_df_B_1.to_csv('real_value_df_B_1.csv', index=False)

In [ ]:
real_value_df_B_3= real_value_df.loc[real_value_df['route'] == 'B_3']
real_value_df_B_3.to_csv('real_value_df_B_3.csv', index=False)

In [ ]:
real_value_df_C_1= real_value_df.loc[real_value_df['route'] == 'C_1']
real_value_df_C_1.to_csv('real_value_df_C_1.csv', index=False)

In [ ]:
real_value_df_C_3= real_value_df.loc[real_value_df['route'] == 'C_3']
real_value_df_C_3.to_csv('real_value_df_C_3.csv', index=False)

## B1

In [230]:
#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
B_1_travel_times=travel_times['B_1']
B_1_travel_times_need={}
for i in B_1_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in B_1_travel_times_need:
            B_1_travel_times_need[i]=[]
        if i in B_1_travel_times_need:
            B_1_travel_times_need[i].extend(B_1_travel_times[i])
        
B_1_ratio_8_9={}
B_1_ratio_9_10={}
B_1_ratio_17_18={}
B_1_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    B_1_ratio_8_9[i]={}
    B_1_ratio_9_10[i]={}
    B_1_ratio_17_18[i]={}
    B_1_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        B_1_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        B_1_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        B_1_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        B_1_ratio_18_19[i][j]=[]

for i in B_1_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        B_1_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        B_1_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        B_1_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        B_1_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
for i in date_key:
    k=0
    if B_1_ratio_8_9[i][480]==[]:k+=1
    if B_1_ratio_8_9[i][500]==[]:k+=1
    if B_1_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del B_1_ratio_8_9[i]
        
for i in date_key:
    k=0
    if B_1_ratio_9_10[i][540]==[]:k+=1
    if B_1_ratio_9_10[i][560]==[]:k+=1
    if B_1_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del B_1_ratio_9_10[i]
        
for i in date_key:
    k=0
    if B_1_ratio_17_18[i][17*60]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del B_1_ratio_17_18[i]
        
for i in date_key:
    k=0
    if B_1_ratio_18_19[i][18*60]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del B_1_ratio_18_19[i]
        
for i in B_1_ratio_8_9:
    for k in [480,500,520]:
        if B_1_ratio_8_9[i][k]==[] and B_1_ratio_8_9[i][k-20]!=[] and B_1_ratio_8_9[i][k+20]!=[]:
            B_1_ratio_8_9[i][k]=[np.mean([np.mean(B_1_ratio_8_9[i][k-20]),np.mean(B_1_ratio_8_9[i][k+20])])]
            
for i in B_1_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if B_1_ratio_9_10[i][k]==[] and B_1_ratio_9_10[i][k-20]!=[] and B_1_ratio_9_10[i][k+20]!=[]:
            B_1_ratio_9_10[i][k]=[np.mean([np.mean(B_1_ratio_9_10[i][k-20]),np.mean(B_1_ratio_9_10[i][k+20])])]
            
for i in B_1_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if B_1_ratio_17_18[i][k]==[] and B_1_ratio_17_18[i][k-20]!=[] and B_1_ratio_17_18[i][k+20]!=[]:
            B_1_ratio_17_18[i][k]=[np.mean([np.mean(B_1_ratio_17_18[i][k-20]),np.mean(B_1_ratio_17_18[i][k+20])])]
            
for i in B_1_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if B_1_ratio_18_19[i][k]==[] and B_1_ratio_18_19[i][k-20]!=[] and B_1_ratio_18_19[i][k+20]!=[]:
            B_1_ratio_18_19[i][k]=[np.mean([np.mean(B_1_ratio_18_19[i][k-20]),np.mean(B_1_ratio_18_19[i][k+20])])]
            
B_1_ratio_8_9_key=list(B_1_ratio_8_9.keys())
for i in B_1_ratio_8_9_key:
    k=0
    if B_1_ratio_8_9[i][480]==[]:k+=1
    if B_1_ratio_8_9[i][500]==[]:k+=1
    if B_1_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del B_1_ratio_8_9[i]
        
B_1_ratio_9_10_key=list(B_1_ratio_9_10.keys())
for i in B_1_ratio_9_10_key:
    k=0
    if B_1_ratio_9_10[i][540]==[]:k+=1
    if B_1_ratio_9_10[i][560]==[]:k+=1
    if B_1_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del B_1_ratio_9_10[i]

B_1_ratio_17_18_key=list(B_1_ratio_17_18.keys())
for i in B_1_ratio_17_18_key:
    k=0
    if B_1_ratio_17_18[i][17*60]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del B_1_ratio_17_18[i]

B_1_ratio_18_19_key=list(B_1_ratio_18_19.keys())
for i in  B_1_ratio_18_19_key:
    k=0
    if B_1_ratio_18_19[i][18*60]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del B_1_ratio_18_19[i]
        
for i in B_1_ratio_8_9:
    del B_1_ratio_8_9[i][8*60-20]
    del B_1_ratio_8_9[i][9*60]
    
for i in B_1_ratio_9_10:
    del B_1_ratio_9_10[i][9*60-20]
    del B_1_ratio_9_10[i][10*60]
    
for i in B_1_ratio_17_18:
    del B_1_ratio_17_18[i][17*60-20]
    del B_1_ratio_17_18[i][18*60]
    
for i in B_1_ratio_18_19:
    del B_1_ratio_18_19[i][18*60-20]
    del B_1_ratio_18_19[i][19*60]
    
for i in B_1_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in B_1_ratio_8_9[i]: 
        if B_1_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_8_9[i][j]))
            temp2.append(np.mean(B_1_ratio_8_9[i][j]))
        if B_1_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in B_1_ratio_9_10[i]: 
        if B_1_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_9_10[i][j]))
            temp2.append(np.mean(B_1_ratio_9_10[i][j]))
        if B_1_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in B_1_ratio_17_18[i]: 
        if B_1_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_17_18[i][j]))
            temp2.append(np.mean(B_1_ratio_17_18[i][j]))
        if B_1_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in B_1_ratio_18_19[i]: 
        if B_1_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_18_19[i][j]))
            temp2.append(np.mean(B_1_ratio_18_19[i][j]))
        if B_1_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
B_1_ratio_8_9_480=[]
B_1_ratio_8_9_500=[]
B_1_ratio_8_9_520=[]

for i in B_1_ratio_8_9:
    for j in B_1_ratio_8_9[i]:
        if j==480: B_1_ratio_8_9_480.extend(B_1_ratio_8_9[i][j])
        if j==500: B_1_ratio_8_9_500.extend(B_1_ratio_8_9[i][j])
        if j==520: B_1_ratio_8_9_520.extend(B_1_ratio_8_9[i][j])
    
    
B_1_ratio_9_10_540=[]
B_1_ratio_9_10_560=[]
B_1_ratio_9_10_580=[]

for i in B_1_ratio_9_10:
    for j in B_1_ratio_9_10[i]:
        if j==540: B_1_ratio_9_10_540.extend(B_1_ratio_9_10[i][j])
        if j==560: B_1_ratio_9_10_560.extend(B_1_ratio_9_10[i][j])
        if j==580: B_1_ratio_9_10_580.extend(B_1_ratio_9_10[i][j])
            
B_1_ratio_17_18_1020=[]
B_1_ratio_17_18_1040=[]
B_1_ratio_17_18_1060=[]

for i in B_1_ratio_17_18:
    for j in B_1_ratio_17_18[i]:
        if j==1020: B_1_ratio_17_18_1020.extend(B_1_ratio_17_18[i][j])
        if j==1040: B_1_ratio_17_18_1040.extend(B_1_ratio_17_18[i][j])
        if j==1060: B_1_ratio_17_18_1060.extend(B_1_ratio_17_18[i][j])
    
B_1_ratio_18_19_1080=[]
B_1_ratio_18_19_1100=[]
B_1_ratio_18_19_1120=[]

for i in B_1_ratio_18_19:
    for j in B_1_ratio_18_19[i]:
        if j==1080: B_1_ratio_18_19_1080.extend(B_1_ratio_18_19[i][j])
        if j==1100: B_1_ratio_18_19_1100.extend(B_1_ratio_18_19[i][j])
        if j==1120: B_1_ratio_18_19_1120.extend(B_1_ratio_18_19[i][j])
            
B_1_ratio_8_9_480.sort()
B_1_ratio_8_9_480_used=B_1_ratio_8_9_480[5:len(B_1_ratio_8_9_480)-5]

B_1_ratio_8_9_500.sort()
B_1_ratio_8_9_500_used=B_1_ratio_8_9_500[5:len(B_1_ratio_8_9_500)-5]

B_1_ratio_8_9_520.sort()
B_1_ratio_8_9_520_used=B_1_ratio_8_9_520[5:len(B_1_ratio_8_9_520)-5]

B_1_ratio_9_10_540.sort()
B_1_ratio_9_10_540_used=B_1_ratio_9_10_540[5:len(B_1_ratio_9_10_540)-5]

B_1_ratio_9_10_560.sort()
B_1_ratio_9_10_560_used=B_1_ratio_9_10_560[5:len(B_1_ratio_9_10_560)-5]

B_1_ratio_9_10_580.sort()
B_1_ratio_9_10_580_used=B_1_ratio_9_10_580[5:len(B_1_ratio_9_10_580)-5]

B_1_ratio_17_18_1020.sort()
B_1_ratio_17_18_1020_used=B_1_ratio_17_18_1020[5:len(B_1_ratio_17_18_1020)-5]

B_1_ratio_17_18_1040.sort()
B_1_ratio_17_18_1040_used=B_1_ratio_17_18_1040[5:len(B_1_ratio_17_18_1040)-5]

B_1_ratio_17_18_1060.sort()
B_1_ratio_17_18_1060_used=B_1_ratio_17_18_1060[5:len(B_1_ratio_17_18_1060)-5]

B_1_ratio_18_19_1080.sort()
B_1_ratio_18_19_1080_used=B_1_ratio_18_19_1080[5:len(B_1_ratio_18_19_1080)-5]

B_1_ratio_18_19_1100.sort()
B_1_ratio_18_19_1100_used=B_1_ratio_18_19_1100[5:len(B_1_ratio_18_19_1100)-5]

B_1_ratio_18_19_1120.sort()
B_1_ratio_18_19_1120_used=B_1_ratio_18_19_1120[5:len(B_1_ratio_18_19_1120)-5]

names=locals()
B_1_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    B_1_ratio.append(np.mean(names['B_1_ratio_'+str(i)+'_used']))
    
B_1_ratio_8=B_1_ratio[0:3]
B_1_ratio_9=B_1_ratio[3:6]
B_1_ratio_17=B_1_ratio[6:9]
B_1_ratio_18=B_1_ratio[9:12]


In [231]:
print(B_1_ratio_8)
print(B_1_ratio_9)
print(B_1_ratio_17)
print(B_1_ratio_18)

[1.0031420374343412, 0.9934482062307293, 0.9873723329424676]
[1.0143332259275053, 0.9560381921561566, 0.9965976774126419]
[0.9289831850456122, 1.000556880529891, 1.0657150770402124]
[1.0174539659659674, 1.0053514813377316, 0.9790868479720023]


In [232]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/B_1.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/B_1.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

#print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#print(test_compare)

train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/B_1.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/B_1.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in B_1_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in B_1_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in B_1_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in B_1_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    


0.14568259200451397


In [233]:
final_result.sort(key = lambda x: x[0]) 

In [234]:
real_value_df_B_1.loc[real_value_df_B_1['route'] == 'B_1']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_B_1_time=list(real_value_df_B_1['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_B_1_time:final_result3.append(i[0])
    

In [235]:
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    

In [236]:


#load the real value 
real_value_df_B_1=real_value_df.loc[real_value_df['route'] =='B_1']

real_value=list(real_value_df_B_1['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))

In [237]:
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.24692491475343145

In [238]:
mape_list.remove(5.374733704123933)

In [239]:
np.mean(mape_list)

0.17570534823439676

In [240]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[88.71, 127.80725530695705],
 [136.07, 126.5721939564116],
 [108.32, 125.79808554545149],
 [109.72, 137.70427874941532],
 [121.11, 129.79023691881665],
 [103.39, 135.29652865897407],
 [120.0, 103.2424137055729],
 [150.77, 111.196746139762],
 [154.23, 118.43809301096218],
 [131.32, 112.43402669214903],
 [131.49, 111.09663834314347],
 [175.32, 108.19425790365271],
 [92.31, 132.60364983990164],
 [135.66, 131.32223868319392],
 [153.98, 130.51907926615974],
 [117.8, 137.18671241745025],
 [137.92, 129.30241578895044],
 [142.04, 134.78801193965657],
 [120.2, 116.64362152944541],
 [115.66, 133.81175042301638],
 [141.59, 135.53748141312667],
 [108.92, 122.30655894992937],
 [138.17, 121.12465340384381],
 [83.41, 120.38386184415234],
 [119.25, 121.68164049366598],
 [105.32, 114.68844027048968],
 [132.5, 119.55404515992325],
 [87.61, 134.1110467724574],
 [173.74, 144.44365922151746],
 [140.4, 153.85010928484732],
 [21.7, 138.33172137948935],
 [143.75, 136.68628326868182],
 [137.51, 133.1153777865

## A2

In [112]:
#A2

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
A_2_travel_times=travel_times['A_2']
A_2_travel_times_need={}
for i in A_2_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in A_2_travel_times_need:
            A_2_travel_times_need[i]=[]
        if i in A_2_travel_times_need:
            A_2_travel_times_need[i].extend(A_2_travel_times[i])
        
A_2_ratio_8_9={}
A_2_ratio_9_10={}
A_2_ratio_17_18={}
A_2_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    A_2_ratio_8_9[i]={}
    A_2_ratio_9_10[i]={}
    A_2_ratio_17_18[i]={}
    A_2_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        A_2_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        A_2_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        A_2_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        A_2_ratio_18_19[i][j]=[]

for i in A_2_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        A_2_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        A_2_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        A_2_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        A_2_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
for i in date_key:
    k=0
    if A_2_ratio_8_9[i][480]==[]:k+=1
    if A_2_ratio_8_9[i][500]==[]:k+=1
    if A_2_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del A_2_ratio_8_9[i]
        
for i in date_key:
    k=0
    if A_2_ratio_9_10[i][540]==[]:k+=1
    if A_2_ratio_9_10[i][560]==[]:k+=1
    if A_2_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del A_2_ratio_9_10[i]
        
for i in date_key:
    k=0
    if A_2_ratio_17_18[i][17*60]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del A_2_ratio_17_18[i]
        
for i in date_key:
    k=0
    if A_2_ratio_18_19[i][18*60]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del A_2_ratio_18_19[i]
        
for i in A_2_ratio_8_9:
    for k in [480,500,520]:
        if A_2_ratio_8_9[i][k]==[] and A_2_ratio_8_9[i][k-20]!=[] and A_2_ratio_8_9[i][k+20]!=[]:
            A_2_ratio_8_9[i][k]=[np.mean([np.mean(A_2_ratio_8_9[i][k-20]),np.mean(A_2_ratio_8_9[i][k+20])])]
            
for i in A_2_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if A_2_ratio_9_10[i][k]==[] and A_2_ratio_9_10[i][k-20]!=[] and A_2_ratio_9_10[i][k+20]!=[]:
            A_2_ratio_9_10[i][k]=[np.mean([np.mean(A_2_ratio_9_10[i][k-20]),np.mean(A_2_ratio_9_10[i][k+20])])]
            
for i in A_2_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if A_2_ratio_17_18[i][k]==[] and A_2_ratio_17_18[i][k-20]!=[] and A_2_ratio_17_18[i][k+20]!=[]:
            A_2_ratio_17_18[i][k]=[np.mean([np.mean(A_2_ratio_17_18[i][k-20]),np.mean(A_2_ratio_17_18[i][k+20])])]
            
for i in A_2_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if A_2_ratio_18_19[i][k]==[] and A_2_ratio_18_19[i][k-20]!=[] and A_2_ratio_18_19[i][k+20]!=[]:
            A_2_ratio_18_19[i][k]=[np.mean([np.mean(A_2_ratio_18_19[i][k-20]),np.mean(A_2_ratio_18_19[i][k+20])])]
            
A_2_ratio_8_9_key=list(A_2_ratio_8_9.keys())
for i in A_2_ratio_8_9_key:
    k=0
    if A_2_ratio_8_9[i][480]==[]:k+=1
    if A_2_ratio_8_9[i][500]==[]:k+=1
    if A_2_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del A_2_ratio_8_9[i]
        
A_2_ratio_9_10_key=list(A_2_ratio_9_10.keys())
for i in A_2_ratio_9_10_key:
    k=0
    if A_2_ratio_9_10[i][540]==[]:k+=1
    if A_2_ratio_9_10[i][560]==[]:k+=1
    if A_2_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del A_2_ratio_9_10[i]

A_2_ratio_17_18_key=list(A_2_ratio_17_18.keys())
for i in A_2_ratio_17_18_key:
    k=0
    if A_2_ratio_17_18[i][17*60]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del A_2_ratio_17_18[i]

A_2_ratio_18_19_key=list(A_2_ratio_18_19.keys())
for i in  A_2_ratio_18_19_key:
    k=0
    if A_2_ratio_18_19[i][18*60]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del A_2_ratio_18_19[i]
        
for i in A_2_ratio_8_9:
    del A_2_ratio_8_9[i][8*60-20]
    del A_2_ratio_8_9[i][9*60]
    
for i in A_2_ratio_9_10:
    del A_2_ratio_9_10[i][9*60-20]
    del A_2_ratio_9_10[i][10*60]
    
for i in A_2_ratio_17_18:
    del A_2_ratio_17_18[i][17*60-20]
    del A_2_ratio_17_18[i][18*60]
    
for i in A_2_ratio_18_19:
    del A_2_ratio_18_19[i][18*60-20]
    del A_2_ratio_18_19[i][19*60]
    
for i in A_2_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in A_2_ratio_8_9[i]: 
        if A_2_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_8_9[i][j]))
            temp2.append(np.mean(A_2_ratio_8_9[i][j]))
        if A_2_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in A_2_ratio_9_10[i]: 
        if A_2_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_9_10[i][j]))
            temp2.append(np.mean(A_2_ratio_9_10[i][j]))
        if A_2_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in A_2_ratio_17_18[i]: 
        if A_2_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_17_18[i][j]))
            temp2.append(np.mean(A_2_ratio_17_18[i][j]))
        if A_2_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in A_2_ratio_18_19[i]: 
        if A_2_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_18_19[i][j]))
            temp2.append(np.mean(A_2_ratio_18_19[i][j]))
        if A_2_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
A_2_ratio_8_9_480=[]
A_2_ratio_8_9_500=[]
A_2_ratio_8_9_520=[]

for i in A_2_ratio_8_9:
    for j in A_2_ratio_8_9[i]:
        if j==480: A_2_ratio_8_9_480.extend(A_2_ratio_8_9[i][j])
        if j==500: A_2_ratio_8_9_500.extend(A_2_ratio_8_9[i][j])
        if j==520: A_2_ratio_8_9_520.extend(A_2_ratio_8_9[i][j])
    
    
A_2_ratio_9_10_540=[]
A_2_ratio_9_10_560=[]
A_2_ratio_9_10_580=[]

for i in A_2_ratio_9_10:
    for j in A_2_ratio_9_10[i]:
        if j==540: A_2_ratio_9_10_540.extend(A_2_ratio_9_10[i][j])
        if j==560: A_2_ratio_9_10_560.extend(A_2_ratio_9_10[i][j])
        if j==580: A_2_ratio_9_10_580.extend(A_2_ratio_9_10[i][j])
            
A_2_ratio_17_18_1020=[]
A_2_ratio_17_18_1040=[]
A_2_ratio_17_18_1060=[]

for i in A_2_ratio_17_18:
    for j in A_2_ratio_17_18[i]:
        if j==1020: A_2_ratio_17_18_1020.extend(A_2_ratio_17_18[i][j])
        if j==1040: A_2_ratio_17_18_1040.extend(A_2_ratio_17_18[i][j])
        if j==1060: A_2_ratio_17_18_1060.extend(A_2_ratio_17_18[i][j])
    
A_2_ratio_18_19_1080=[]
A_2_ratio_18_19_1100=[]
A_2_ratio_18_19_1120=[]

for i in A_2_ratio_18_19:
    for j in A_2_ratio_18_19[i]:
        if j==1080: A_2_ratio_18_19_1080.extend(A_2_ratio_18_19[i][j])
        if j==1100: A_2_ratio_18_19_1100.extend(A_2_ratio_18_19[i][j])
        if j==1120: A_2_ratio_18_19_1120.extend(A_2_ratio_18_19[i][j])
            
A_2_ratio_8_9_480.sort()
A_2_ratio_8_9_480_used=A_2_ratio_8_9_480[5:len(A_2_ratio_8_9_480)-5]

A_2_ratio_8_9_500.sort()
A_2_ratio_8_9_500_used=A_2_ratio_8_9_500[5:len(A_2_ratio_8_9_500)-5]

A_2_ratio_8_9_520.sort()
A_2_ratio_8_9_520_used=A_2_ratio_8_9_520[5:len(A_2_ratio_8_9_520)-5]

A_2_ratio_9_10_540.sort()
A_2_ratio_9_10_540_used=A_2_ratio_9_10_540[5:len(A_2_ratio_9_10_540)-5]

A_2_ratio_9_10_560.sort()
A_2_ratio_9_10_560_used=A_2_ratio_9_10_560[5:len(A_2_ratio_9_10_560)-5]

A_2_ratio_9_10_580.sort()
A_2_ratio_9_10_580_used=A_2_ratio_9_10_580[5:len(A_2_ratio_9_10_580)-5]

A_2_ratio_17_18_1020.sort()
A_2_ratio_17_18_1020_used=A_2_ratio_17_18_1020[5:len(A_2_ratio_17_18_1020)-5]

A_2_ratio_17_18_1040.sort()
A_2_ratio_17_18_1040_used=A_2_ratio_17_18_1040[5:len(A_2_ratio_17_18_1040)-5]

A_2_ratio_17_18_1060.sort()
A_2_ratio_17_18_1060_used=A_2_ratio_17_18_1060[5:len(A_2_ratio_17_18_1060)-5]

A_2_ratio_18_19_1080.sort()
A_2_ratio_18_19_1080_used=A_2_ratio_18_19_1080[5:len(A_2_ratio_18_19_1080)-5]

A_2_ratio_18_19_1100.sort()
A_2_ratio_18_19_1100_used=A_2_ratio_18_19_1100[5:len(A_2_ratio_18_19_1100)-5]

A_2_ratio_18_19_1120.sort()
A_2_ratio_18_19_1120_used=A_2_ratio_18_19_1120[5:len(A_2_ratio_18_19_1120)-5]

names=locals()
A_2_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    A_2_ratio.append(np.mean(names['A_2_ratio_'+str(i)+'_used']))
    
A_2_ratio_8=A_2_ratio[0:3]
A_2_ratio_9=A_2_ratio[3:6]
A_2_ratio_17=A_2_ratio[6:9]
A_2_ratio_18=A_2_ratio[9:12]






In [115]:
print(A_2_ratio_8)
print(A_2_ratio_9)
print(A_2_ratio_17)
print(A_2_ratio_18)


[0.9594377262857952, 1.0144147364368832, 1.004964586594884]
[1.000155698203997, 1.0194005415982434, 0.9602155822869323]
[1.0152742931390726, 0.9915728865263712, 0.9819021063140361]
[1.0065583872672326, 0.969453070970589, 1.0087499022367605]


In [114]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/A_2.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/A_2.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

#print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#print(test_compare)

train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/A_2.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/A_2.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in A_2_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in A_2_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in A_2_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in A_2_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])

0.09358656422206517


In [119]:
final_result.sort(key = lambda x: x[0]) 

In [120]:
final_result

[['10-18', 8, 82.44221939830102],
 ['10-18', 8, 87.16626412633636],
 ['10-18', 8, 86.35423505422872],
 ['10-18', 9, 76.39502671813483],
 ['10-18', 9, 77.86500816995266],
 ['10-18', 9, 73.34427549201202],
 ['10-18', 17, 77.72956328786886],
 ['10-18', 17, 75.91497978293505],
 ['10-18', 17, 75.17458329339762],
 ['10-18', 18, 78.4388443881954],
 ['10-18', 18, 75.54731005915455],
 ['10-18', 18, 78.60962424939741],
 ['10-19', 8, 98.96720846762604],
 ['10-19', 8, 104.63815622742759],
 ['10-19', 8, 103.6633613826553],
 ['10-19', 9, 79.25959149316411],
 ['10-19', 9, 80.78469246346002],
 ['10-19', 9, 76.09444702869645],
 ['10-19', 17, 68.0370839622574],
 ['10-19', 17, 66.44876974743953],
 ['10-19', 17, 65.80069691654687],
 ['10-19', 18, 69.79375269646481],
 ['10-19', 18, 67.2209071446404],
 ['10-19', 18, 69.94571015442122],
 ['10-20', 8, 89.31857018843752],
 ['10-20', 8, 94.43663862101829],
 ['10-20', 8, 93.55687972804515],
 ['10-20', 9, 78.03214298788075],
 ['10-20', 9, 79.53362558125684],
 ['1

In [133]:
final_result1=final_result

In [134]:
pred_value=[]
for i in final_result1:
    pred_value.append(i[2])




In [137]:
#load the real value 
real_value_df_A_2=real_value_df.loc[real_value_df['route'] =='A_2']

real_value=list(real_value_df_A_2['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))

In [138]:
real_value1

[77.77,
 70.3,
 77.68,
 79.28,
 83.27,
 77.67,
 75.0,
 63.3,
 68.39,
 88.83,
 59.33,
 52.23,
 91.72,
 92.28,
 103.94,
 79.27,
 81.22,
 79.82,
 60.0,
 56.98,
 66.91,
 63.8,
 55.74,
 84.02,
 87.24,
 75.48,
 79.6,
 103.35,
 97.05,
 67.95,
 96.52,
 54.93,
 82.28,
 75.25,
 50.34,
 57.25,
 58.58,
 71.5,
 55.92,
 99.53,
 83.67,
 73.3,
 101.05,
 82.62,
 62.0,
 76.75,
 81.04,
 70.41,
 71.43,
 56.06,
 62.2,
 79.54,
 64.81,
 70.94,
 71.32,
 63.61,
 77.17,
 69.61,
 66.35,
 58.64,
 56.62,
 65.06,
 65.96,
 83.79,
 64.03,
 66.64,
 94.08,
 67.87,
 63.09,
 63.97,
 60.32,
 69.44,
 70.59,
 85.32,
 63.33,
 79.18,
 76.7,
 51.01,
 85.03,
 68.98,
 53.84,
 65.13,
 61.35,
 62.85]

In [139]:
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.133475149336903

In [140]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[77.77, 82.44221939830102],
 [70.3, 87.16626412633636],
 [77.68, 86.35423505422872],
 [79.28, 76.39502671813483],
 [83.27, 77.86500816995266],
 [77.67, 73.34427549201202],
 [75.0, 77.72956328786886],
 [63.3, 75.91497978293505],
 [68.39, 75.17458329339762],
 [88.83, 78.4388443881954],
 [59.33, 75.54731005915455],
 [52.23, 78.60962424939741],
 [91.72, 98.96720846762604],
 [92.28, 104.63815622742759],
 [103.94, 103.6633613826553],
 [79.27, 79.25959149316411],
 [81.22, 80.78469246346002],
 [79.82, 76.09444702869645],
 [60.0, 68.0370839622574],
 [56.98, 66.44876974743953],
 [66.91, 65.80069691654687],
 [63.8, 69.79375269646481],
 [55.74, 67.2209071446404],
 [84.02, 69.94571015442122],
 [87.24, 89.31857018843752],
 [75.48, 94.43663862101829],
 [79.6, 93.55687972804515],
 [103.35, 78.03214298788075],
 [97.05, 79.53362558125684],
 [67.95, 74.91601532716801],
 [96.52, 69.4718611889569],
 [54.93, 67.85005234251241],
 [82.28, 67.18831284507768],
 [75.25, 62.648695888729954],
 [50.34, 60.33924250

## A3

In [144]:
#A3

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
A_3_travel_times=travel_times['A_3']
A_3_travel_times_need={}
for i in A_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in A_3_travel_times_need:
            A_3_travel_times_need[i]=[]
        if i in A_3_travel_times_need:
            A_3_travel_times_need[i].extend(A_3_travel_times[i])
        
A_3_ratio_8_9={}
A_3_ratio_9_10={}
A_3_ratio_17_18={}
A_3_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    A_3_ratio_8_9[i]={}
    A_3_ratio_9_10[i]={}
    A_3_ratio_17_18[i]={}
    A_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        A_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        A_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        A_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        A_3_ratio_18_19[i][j]=[]

for i in A_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        A_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        A_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        A_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        A_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if A_3_ratio_8_9[i][480]==[]:k+=1
    if A_3_ratio_8_9[i][500]==[]:k+=1
    if A_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del A_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if A_3_ratio_9_10[i][540]==[]:k+=1
    if A_3_ratio_9_10[i][560]==[]:k+=1
    if A_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del A_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if A_3_ratio_17_18[i][17*60]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del A_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if A_3_ratio_18_19[i][18*60]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del A_3_ratio_18_19[i]
        
for i in A_3_ratio_8_9:
    for k in [480,500,520]:
        if A_3_ratio_8_9[i][k]==[] and A_3_ratio_8_9[i][k-20]!=[] and A_3_ratio_8_9[i][k+20]!=[]:
            A_3_ratio_8_9[i][k]=[np.mean([np.mean(A_3_ratio_8_9[i][k-20]),np.mean(A_3_ratio_8_9[i][k+20])])]
            
for i in A_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if A_3_ratio_9_10[i][k]==[] and A_3_ratio_9_10[i][k-20]!=[] and A_3_ratio_9_10[i][k+20]!=[]:
            A_3_ratio_9_10[i][k]=[np.mean([np.mean(A_3_ratio_9_10[i][k-20]),np.mean(A_3_ratio_9_10[i][k+20])])]
            
for i in A_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if A_3_ratio_17_18[i][k]==[] and A_3_ratio_17_18[i][k-20]!=[] and A_3_ratio_17_18[i][k+20]!=[]:
            A_3_ratio_17_18[i][k]=[np.mean([np.mean(A_3_ratio_17_18[i][k-20]),np.mean(A_3_ratio_17_18[i][k+20])])]
            
for i in A_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if A_3_ratio_18_19[i][k]==[] and A_3_ratio_18_19[i][k-20]!=[] and A_3_ratio_18_19[i][k+20]!=[]:
            A_3_ratio_18_19[i][k]=[np.mean([np.mean(A_3_ratio_18_19[i][k-20]),np.mean(A_3_ratio_18_19[i][k+20])])]
            
A_3_ratio_8_9_key=list(A_3_ratio_8_9.keys())
for i in A_3_ratio_8_9_key:
    k=0
    if A_3_ratio_8_9[i][480]==[]:k+=1
    if A_3_ratio_8_9[i][500]==[]:k+=1
    if A_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del A_3_ratio_8_9[i]
        
A_3_ratio_9_10_key=list(A_3_ratio_9_10.keys())
for i in A_3_ratio_9_10_key:
    k=0
    if A_3_ratio_9_10[i][540]==[]:k+=1
    if A_3_ratio_9_10[i][560]==[]:k+=1
    if A_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del A_3_ratio_9_10[i]

A_3_ratio_17_18_key=list(A_3_ratio_17_18.keys())
for i in A_3_ratio_17_18_key:
    k=0
    if A_3_ratio_17_18[i][17*60]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del A_3_ratio_17_18[i]

A_3_ratio_18_19_key=list(A_3_ratio_18_19.keys())
for i in  A_3_ratio_18_19_key:
    k=0
    if A_3_ratio_18_19[i][18*60]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del A_3_ratio_18_19[i]
        
for i in A_3_ratio_8_9:
    del A_3_ratio_8_9[i][8*60-20]
    del A_3_ratio_8_9[i][9*60]
    
for i in A_3_ratio_9_10:
    del A_3_ratio_9_10[i][9*60-20]
    del A_3_ratio_9_10[i][10*60]
    
for i in A_3_ratio_17_18:
    del A_3_ratio_17_18[i][17*60-20]
    del A_3_ratio_17_18[i][18*60]
    
for i in A_3_ratio_18_19:
    del A_3_ratio_18_19[i][18*60-20]
    del A_3_ratio_18_19[i][19*60]
    
for i in A_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in A_3_ratio_8_9[i]: 
        if A_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_8_9[i][j]))
            temp2.append(np.mean(A_3_ratio_8_9[i][j]))
        if A_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in A_3_ratio_9_10[i]: 
        if A_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_9_10[i][j]))
            temp2.append(np.mean(A_3_ratio_9_10[i][j]))
        if A_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in A_3_ratio_17_18[i]: 
        if A_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_17_18[i][j]))
            temp2.append(np.mean(A_3_ratio_17_18[i][j]))
        if A_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in A_3_ratio_18_19[i]: 
        if A_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_18_19[i][j]))
            temp2.append(np.mean(A_3_ratio_18_19[i][j]))
        if A_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
A_3_ratio_8_9_480=[]
A_3_ratio_8_9_500=[]
A_3_ratio_8_9_520=[]

for i in A_3_ratio_8_9:
    for j in A_3_ratio_8_9[i]:
        if j==480: A_3_ratio_8_9_480.extend(A_3_ratio_8_9[i][j])
        if j==500: A_3_ratio_8_9_500.extend(A_3_ratio_8_9[i][j])
        if j==520: A_3_ratio_8_9_520.extend(A_3_ratio_8_9[i][j])
    
    
A_3_ratio_9_10_540=[]
A_3_ratio_9_10_560=[]
A_3_ratio_9_10_580=[]

for i in A_3_ratio_9_10:
    for j in A_3_ratio_9_10[i]:
        if j==540: A_3_ratio_9_10_540.extend(A_3_ratio_9_10[i][j])
        if j==560: A_3_ratio_9_10_560.extend(A_3_ratio_9_10[i][j])
        if j==580: A_3_ratio_9_10_580.extend(A_3_ratio_9_10[i][j])
            
A_3_ratio_17_18_1020=[]
A_3_ratio_17_18_1040=[]
A_3_ratio_17_18_1060=[]

for i in A_3_ratio_17_18:
    for j in A_3_ratio_17_18[i]:
        if j==1020: A_3_ratio_17_18_1020.extend(A_3_ratio_17_18[i][j])
        if j==1040: A_3_ratio_17_18_1040.extend(A_3_ratio_17_18[i][j])
        if j==1060: A_3_ratio_17_18_1060.extend(A_3_ratio_17_18[i][j])
    
A_3_ratio_18_19_1080=[]
A_3_ratio_18_19_1100=[]
A_3_ratio_18_19_1120=[]

for i in A_3_ratio_18_19:
    for j in A_3_ratio_18_19[i]:
        if j==1080: A_3_ratio_18_19_1080.extend(A_3_ratio_18_19[i][j])
        if j==1100: A_3_ratio_18_19_1100.extend(A_3_ratio_18_19[i][j])
        if j==1120: A_3_ratio_18_19_1120.extend(A_3_ratio_18_19[i][j])
            
A_3_ratio_8_9_480.sort()
A_3_ratio_8_9_480_used=A_3_ratio_8_9_480[5:len(A_3_ratio_8_9_480)-5]

A_3_ratio_8_9_500.sort()
A_3_ratio_8_9_500_used=A_3_ratio_8_9_500[5:len(A_3_ratio_8_9_500)-5]

A_3_ratio_8_9_520.sort()
A_3_ratio_8_9_520_used=A_3_ratio_8_9_520[5:len(A_3_ratio_8_9_520)-5]

A_3_ratio_9_10_540.sort()
A_3_ratio_9_10_540_used=A_3_ratio_9_10_540[5:len(A_3_ratio_9_10_540)-5]

A_3_ratio_9_10_560.sort()
A_3_ratio_9_10_560_used=A_3_ratio_9_10_560[5:len(A_3_ratio_9_10_560)-5]

A_3_ratio_9_10_580.sort()
A_3_ratio_9_10_580_used=A_3_ratio_9_10_580[5:len(A_3_ratio_9_10_580)-5]

A_3_ratio_17_18_1020.sort()
A_3_ratio_17_18_1020_used=A_3_ratio_17_18_1020[5:len(A_3_ratio_17_18_1020)-5]

A_3_ratio_17_18_1040.sort()
A_3_ratio_17_18_1040_used=A_3_ratio_17_18_1040[5:len(A_3_ratio_17_18_1040)-5]

A_3_ratio_17_18_1060.sort()
A_3_ratio_17_18_1060_used=A_3_ratio_17_18_1060[5:len(A_3_ratio_17_18_1060)-5]

A_3_ratio_18_19_1080.sort()
A_3_ratio_18_19_1080_used=A_3_ratio_18_19_1080[5:len(A_3_ratio_18_19_1080)-5]

A_3_ratio_18_19_1100.sort()
A_3_ratio_18_19_1100_used=A_3_ratio_18_19_1100[5:len(A_3_ratio_18_19_1100)-5]

A_3_ratio_18_19_1120.sort()
A_3_ratio_18_19_1120_used=A_3_ratio_18_19_1120[5:len(A_3_ratio_18_19_1120)-5]

names=locals()
A_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    A_3_ratio.append(np.mean(names['A_3_ratio_'+str(i)+'_used']))
    
A_3_ratio_8=A_3_ratio[0:3]
A_3_ratio_9=A_3_ratio[3:6]
A_3_ratio_17=A_3_ratio[6:9]
A_3_ratio_18=A_3_ratio[9:12]





In [145]:
print(A_3_ratio_8)
print(A_3_ratio_9)
print(A_3_ratio_17)
print(A_3_ratio_18)

[0.9702602614629005, 0.978968720287374, 1.0444125878050396]
[1.057196991804728, 1.0448782719176082, 0.8779080952581202]
[1.0131716009078262, 1.008282644002323, 0.977339452004021]
[0.9618902905147334, 1.0074499810834463, 1.0076270414107436]


In [146]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/A_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/A_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=28, learning_rate=0.1, 
                                 max_depth=6, random_state=0, loss='ls').fit(train_x, train_y)

#print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#print(test_compare)

train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/A_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/A_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in A_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in A_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in A_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in A_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0]) 

0.1630436689624688


In [147]:
final_result

[['10-18', 8, 205.8902948984465],
 ['10-18', 8, 207.7382394414689],
 ['10-18', 8, 221.62550012572217],
 ['10-18', 9, 226.86798487526028],
 ['10-18', 9, 224.22446320550713],
 ['10-18', 9, 188.39368823485623],
 ['10-18', 17, 136.28690841999193],
 ['10-18', 17, 135.62926975201842],
 ['10-18', 17, 131.4669422940477],
 ['10-18', 18, 130.17800785157345],
 ['10-18', 18, 136.34385630129157],
 ['10-18', 18, 136.36781886844128],
 ['10-19', 8, 233.28152146564813],
 ['10-19', 8, 235.3753127965753],
 ['10-19', 8, 251.11010643029468],
 ['10-19', 9, 250.19178541261485],
 ['10-19', 9, 247.2764890710172],
 ['10-19', 9, 207.76203061821346],
 ['10-19', 17, 124.32741492479265],
 ['10-19', 17, 123.7274855809428],
 ['10-19', 17, 119.93041204747291],
 ['10-19', 18, 122.1918655530961],
 ['10-19', 18, 127.97945239071117],
 ['10-19', 18, 128.00194490562816],
 ['10-20', 8, 158.11280785876633],
 ['10-20', 8, 159.53193108945905],
 ['10-20', 8, 170.19660948693752],
 ['10-20', 9, 183.96683486108827],
 ['10-20', 9, 1

In [148]:
final_result1=final_result
pred_value=[]
for i in final_result1:
    pred_value.append(i[2])

In [149]:
#load the real value 
real_value_df_A_3=real_value_df.loc[real_value_df['route'] =='A_3']

real_value=list(real_value_df_A_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.2152616858735426

In [150]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare


[[197.5, 205.8902948984465],
 [166.9, 207.7382394414689],
 [253.82, 221.62550012572217],
 [240.16, 226.86798487526028],
 [179.19, 224.22446320550713],
 [136.98, 188.39368823485623],
 [136.34, 136.28690841999193],
 [143.7, 135.62926975201842],
 [100.13, 131.4669422940477],
 [122.91, 130.17800785157345],
 [112.75, 136.34385630129157],
 [120.58, 136.36781886844128],
 [319.67, 233.28152146564813],
 [328.02, 235.3753127965753],
 [326.7, 251.11010643029468],
 [339.07, 250.19178541261485],
 [316.84, 247.2764890710172],
 [199.89, 207.76203061821346],
 [152.27, 124.32741492479265],
 [123.18, 123.7274855809428],
 [112.64, 119.93041204747291],
 [110.19, 122.1918655530961],
 [100.43, 127.97945239071117],
 [150.64, 128.00194490562816],
 [183.76, 158.11280785876633],
 [242.81, 159.53193108945905],
 [224.49, 170.19660948693752],
 [201.63, 183.96683486108827],
 [263.23, 181.82320796398076],
 [182.37, 152.76809793779142],
 [139.66, 133.32668330299862],
 [126.32, 132.68332890139678],
 [124.92, 128.61140

In [286]:
#B3

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
B_3_travel_times=travel_times['B_3']
B_3_travel_times_need={}
for i in B_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in B_3_travel_times_need:
            B_3_travel_times_need[i]=[]
        if i in B_3_travel_times_need:
            B_3_travel_times_need[i].extend(B_3_travel_times[i])
        
B_3_ratio_8_9={}
B_3_ratio_9_10={}
B_3_ratio_17_18={}
B_3_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    B_3_ratio_8_9[i]={}
    B_3_ratio_9_10[i]={}
    B_3_ratio_17_18[i]={}
    B_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        B_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        B_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        B_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        B_3_ratio_18_19[i][j]=[]

for i in B_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        B_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        B_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        B_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        B_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if B_3_ratio_8_9[i][480]==[]:k+=1
    if B_3_ratio_8_9[i][500]==[]:k+=1
    if B_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del B_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if B_3_ratio_9_10[i][540]==[]:k+=1
    if B_3_ratio_9_10[i][560]==[]:k+=1
    if B_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del B_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if B_3_ratio_17_18[i][17*60]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del B_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if B_3_ratio_18_19[i][18*60]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del B_3_ratio_18_19[i]
        
for i in B_3_ratio_8_9:
    for k in [480,500,520]:
        if B_3_ratio_8_9[i][k]==[] and B_3_ratio_8_9[i][k-20]!=[] and B_3_ratio_8_9[i][k+20]!=[]:
            B_3_ratio_8_9[i][k]=[np.mean([np.mean(B_3_ratio_8_9[i][k-20]),np.mean(B_3_ratio_8_9[i][k+20])])]
            
for i in B_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if B_3_ratio_9_10[i][k]==[] and B_3_ratio_9_10[i][k-20]!=[] and B_3_ratio_9_10[i][k+20]!=[]:
            B_3_ratio_9_10[i][k]=[np.mean([np.mean(B_3_ratio_9_10[i][k-20]),np.mean(B_3_ratio_9_10[i][k+20])])]
            
for i in B_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if B_3_ratio_17_18[i][k]==[] and B_3_ratio_17_18[i][k-20]!=[] and B_3_ratio_17_18[i][k+20]!=[]:
            B_3_ratio_17_18[i][k]=[np.mean([np.mean(B_3_ratio_17_18[i][k-20]),np.mean(B_3_ratio_17_18[i][k+20])])]
            
for i in B_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if B_3_ratio_18_19[i][k]==[] and B_3_ratio_18_19[i][k-20]!=[] and B_3_ratio_18_19[i][k+20]!=[]:
            B_3_ratio_18_19[i][k]=[np.mean([np.mean(B_3_ratio_18_19[i][k-20]),np.mean(B_3_ratio_18_19[i][k+20])])]
            
B_3_ratio_8_9_key=list(B_3_ratio_8_9.keys())
for i in B_3_ratio_8_9_key:
    k=0
    if B_3_ratio_8_9[i][480]==[]:k+=1
    if B_3_ratio_8_9[i][500]==[]:k+=1
    if B_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del B_3_ratio_8_9[i]
        
B_3_ratio_9_10_key=list(B_3_ratio_9_10.keys())
for i in B_3_ratio_9_10_key:
    k=0
    if B_3_ratio_9_10[i][540]==[]:k+=1
    if B_3_ratio_9_10[i][560]==[]:k+=1
    if B_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del B_3_ratio_9_10[i]

B_3_ratio_17_18_key=list(B_3_ratio_17_18.keys())
for i in B_3_ratio_17_18_key:
    k=0
    if B_3_ratio_17_18[i][17*60]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del B_3_ratio_17_18[i]

B_3_ratio_18_19_key=list(B_3_ratio_18_19.keys())
for i in  B_3_ratio_18_19_key:
    k=0
    if B_3_ratio_18_19[i][18*60]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del B_3_ratio_18_19[i]
        
for i in B_3_ratio_8_9:
    del B_3_ratio_8_9[i][8*60-20]
    del B_3_ratio_8_9[i][9*60]
    
for i in B_3_ratio_9_10:
    del B_3_ratio_9_10[i][9*60-20]
    del B_3_ratio_9_10[i][10*60]
    
for i in B_3_ratio_17_18:
    del B_3_ratio_17_18[i][17*60-20]
    del B_3_ratio_17_18[i][18*60]
    
for i in B_3_ratio_18_19:
    del B_3_ratio_18_19[i][18*60-20]
    del B_3_ratio_18_19[i][19*60]
    
for i in B_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in B_3_ratio_8_9[i]: 
        if B_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_8_9[i][j]))
            temp2.append(np.mean(B_3_ratio_8_9[i][j]))
        if B_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in B_3_ratio_9_10[i]: 
        if B_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_9_10[i][j]))
            temp2.append(np.mean(B_3_ratio_9_10[i][j]))
        if B_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in B_3_ratio_17_18[i]: 
        if B_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_17_18[i][j]))
            temp2.append(np.mean(B_3_ratio_17_18[i][j]))
        if B_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in B_3_ratio_18_19[i]: 
        if B_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_18_19[i][j]))
            temp2.append(np.mean(B_3_ratio_18_19[i][j]))
        if B_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
B_3_ratio_8_9_480=[]
B_3_ratio_8_9_500=[]
B_3_ratio_8_9_520=[]

for i in B_3_ratio_8_9:
    for j in B_3_ratio_8_9[i]:
        if j==480: B_3_ratio_8_9_480.extend(B_3_ratio_8_9[i][j])
        if j==500: B_3_ratio_8_9_500.extend(B_3_ratio_8_9[i][j])
        if j==520: B_3_ratio_8_9_520.extend(B_3_ratio_8_9[i][j])
    
    
B_3_ratio_9_10_540=[]
B_3_ratio_9_10_560=[]
B_3_ratio_9_10_580=[]

for i in B_3_ratio_9_10:
    for j in B_3_ratio_9_10[i]:
        if j==540: B_3_ratio_9_10_540.extend(B_3_ratio_9_10[i][j])
        if j==560: B_3_ratio_9_10_560.extend(B_3_ratio_9_10[i][j])
        if j==580: B_3_ratio_9_10_580.extend(B_3_ratio_9_10[i][j])
            
B_3_ratio_17_18_1020=[]
B_3_ratio_17_18_1040=[]
B_3_ratio_17_18_1060=[]

for i in B_3_ratio_17_18:
    for j in B_3_ratio_17_18[i]:
        if j==1020: B_3_ratio_17_18_1020.extend(B_3_ratio_17_18[i][j])
        if j==1040: B_3_ratio_17_18_1040.extend(B_3_ratio_17_18[i][j])
        if j==1060: B_3_ratio_17_18_1060.extend(B_3_ratio_17_18[i][j])
    
B_3_ratio_18_19_1080=[]
B_3_ratio_18_19_1100=[]
B_3_ratio_18_19_1120=[]

for i in B_3_ratio_18_19:
    for j in B_3_ratio_18_19[i]:
        if j==1080: B_3_ratio_18_19_1080.extend(B_3_ratio_18_19[i][j])
        if j==1100: B_3_ratio_18_19_1100.extend(B_3_ratio_18_19[i][j])
        if j==1120: B_3_ratio_18_19_1120.extend(B_3_ratio_18_19[i][j])
            
B_3_ratio_8_9_480.sort()
B_3_ratio_8_9_480_used=B_3_ratio_8_9_480[5:len(B_3_ratio_8_9_480)-5]

B_3_ratio_8_9_500.sort()
B_3_ratio_8_9_500_used=B_3_ratio_8_9_500[5:len(B_3_ratio_8_9_500)-5]

B_3_ratio_8_9_520.sort()
B_3_ratio_8_9_520_used=B_3_ratio_8_9_520[5:len(B_3_ratio_8_9_520)-5]

B_3_ratio_9_10_540.sort()
B_3_ratio_9_10_540_used=B_3_ratio_9_10_540[5:len(B_3_ratio_9_10_540)-5]

B_3_ratio_9_10_560.sort()
B_3_ratio_9_10_560_used=B_3_ratio_9_10_560[5:len(B_3_ratio_9_10_560)-5]

B_3_ratio_9_10_580.sort()
B_3_ratio_9_10_580_used=B_3_ratio_9_10_580[5:len(B_3_ratio_9_10_580)-5]

B_3_ratio_17_18_1020.sort()
B_3_ratio_17_18_1020_used=B_3_ratio_17_18_1020[5:len(B_3_ratio_17_18_1020)-5]

B_3_ratio_17_18_1040.sort()
B_3_ratio_17_18_1040_used=B_3_ratio_17_18_1040[5:len(B_3_ratio_17_18_1040)-5]

B_3_ratio_17_18_1060.sort()
B_3_ratio_17_18_1060_used=B_3_ratio_17_18_1060[5:len(B_3_ratio_17_18_1060)-5]

B_3_ratio_18_19_1080.sort()
B_3_ratio_18_19_1080_used=B_3_ratio_18_19_1080[5:len(B_3_ratio_18_19_1080)-5]

B_3_ratio_18_19_1100.sort()
B_3_ratio_18_19_1100_used=B_3_ratio_18_19_1100[5:len(B_3_ratio_18_19_1100)-5]

B_3_ratio_18_19_1120.sort()
B_3_ratio_18_19_1120_used=B_3_ratio_18_19_1120[5:len(B_3_ratio_18_19_1120)-5]

names=locals()
B_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    B_3_ratio.append(np.mean(names['B_3_ratio_'+str(i)+'_used']))
    
B_3_ratio_8=B_3_ratio[0:3]
B_3_ratio_9=B_3_ratio[3:6]
B_3_ratio_17=B_3_ratio[6:9]
B_3_ratio_18=B_3_ratio[9:12]




In [287]:
print(B_3_ratio_8)
print(B_3_ratio_9)
print(B_3_ratio_17)
print(B_3_ratio_18)

[0.9588587319159432, 0.9877176009992981, 1.0498548948656439]
[1.0254018000740626, 1.0251925941467648, 0.9276552578492043]
[0.9887454541008326, 0.9964707338380407, 1.0048398973940167]
[1.0134617557170276, 0.9490581825784564, 1.0192924016085947]


In [288]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/B_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/B_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

#print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#print(test_compare)

train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/B_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/B_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in B_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in B_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in B_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in B_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0])

0.14524607907523132


In [289]:
final_result

[['10-18', 8, 98.85493230726598],
 ['10-18', 8, 101.83017929072818],
 ['10-18', 8, 108.23631376544952],
 ['10-18', 9, 110.21469211177342],
 ['10-18', 9, 110.19220573924763],
 ['10-18', 9, 99.70846415749726],
 ['10-18', 17, 102.47639263304252],
 ['10-18', 17, 103.27706260959394],
 ['10-18', 17, 104.14446653748585],
 ['10-18', 18, 109.76881455340525],
 ['10-18', 18, 102.7932145008677],
 ['10-18', 18, 110.40033625019112],
 ['10-19', 8, 91.79771917393968],
 ['10-19', 8, 94.5605644937063],
 ['10-19', 8, 100.50936765178339],
 ['10-19', 9, 102.10365042783124],
 ['10-19', 9, 102.08281889733587],
 ['10-19', 9, 92.37060843674509],
 ['10-19', 17, 102.65383943065054],
 ['10-19', 17, 103.45589583698963],
 ['10-19', 17, 104.32480175031723],
 ['10-19', 18, 111.83093895535063],
 ['10-19', 18, 104.72429480668184],
 ['10-19', 18, 112.47432446160369],
 ['10-20', 8, 103.83988143881308],
 ['10-20', 8, 106.96516094488375],
 ['10-20', 8, 113.69433700934667],
 ['10-20', 9, 115.56475929499359],
 ['10-20', 9, 1

In [290]:
final_result1=final_result

In [291]:
pred_value=[]
for i in final_result1:
    pred_value.append(i[2])
    

#load the real value 
real_value_df_B_3=real_value_df.loc[real_value_df['route'] =='B_3']

real_value=list(real_value_df_B_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.21764746772789725

In [293]:
for i in mape_list:
    if i>1:
        mape_list.remove(i)

np.mean(mape_list)

0.20489378899015584

In [295]:
real_value_df_B_3

,route,time,label
0,B_3,2016-10-18 08:00:00,126.4
1,B_3,2016-10-18 08:20:00,115.52
2,B_3,2016-10-18 08:40:00,98.86
3,B_3,2016-10-18 09:00:00,131.15
4,B_3,2016-10-18 09:20:00,151.95
...,...,...,...
79,B_3,2016-10-24 17:20:00,57.83
80,B_3,2016-10-24 17:40:00,118.39
81,B_3,2016-10-24 18:00:00,104.99
82,B_3,2016-10-24 18:20:00,129.71


In [159]:

test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare


[[126.4, 98.85493230726598],
 [115.52, 101.83017929072818],
 [98.86, 108.23631376544952],
 [131.15, 110.21469211177342],
 [151.95, 110.19220573924763],
 [117.69, 99.70846415749726],
 [80.49, 102.47639263304252],
 [97.56, 103.27706260959394],
 [104.47, 104.14446653748585],
 [125.47, 109.76881455340525],
 [111.18, 102.7932145008677],
 [117.39, 110.40033625019112],
 [70.3, 91.79771917393968],
 [145.63, 94.5605644937063],
 [120.0, 100.50936765178339],
 [129.17, 102.10365042783124],
 [108.18, 102.08281889733587],
 [112.14, 92.37060843674509],
 [106.34, 102.65383943065054],
 [105.73, 103.45589583698963],
 [101.06, 104.32480175031723],
 [87.42, 111.83093895535063],
 [110.38, 104.72429480668184],
 [80.84, 112.47432446160369],
 [93.88, 103.83988143881308],
 [87.78, 106.96516094488375],
 [103.01, 113.69433700934667],
 [94.06, 115.56475929499359],
 [114.88, 115.54118138374992],
 [70.85, 104.54853558315908],
 [116.79, 135.2560432480628],
 [93.09, 136.3128276468234],
 [87.7, 137.45769252906612],
 [

In [280]:
#C1

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
C_1_travel_times=travel_times['C_1']
C_1_travel_times_need={}
for i in C_1_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in C_1_travel_times_need:
            C_1_travel_times_need[i]=[]
        if i in C_1_travel_times_need:
            C_1_travel_times_need[i].extend(C_1_travel_times[i])
        
C_1_ratio_8_9={}
C_1_ratio_9_10={}
C_1_ratio_17_18={}
C_1_ratio_18_19={}


date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    C_1_ratio_8_9[i]={}
    C_1_ratio_9_10[i]={}
    C_1_ratio_17_18[i]={}
    C_1_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        C_1_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        C_1_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        C_1_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        C_1_ratio_18_19[i][j]=[]

for i in C_1_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        C_1_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        C_1_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        C_1_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        C_1_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
for i in date_key:
    k=0
    if C_1_ratio_8_9[i][480]==[]:k+=1
    if C_1_ratio_8_9[i][500]==[]:k+=1
    if C_1_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del C_1_ratio_8_9[i]
        
for i in date_key:
    k=0
    if C_1_ratio_9_10[i][540]==[]:k+=1
    if C_1_ratio_9_10[i][560]==[]:k+=1
    if C_1_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del C_1_ratio_9_10[i]
        
for i in date_key:
    k=0
    if C_1_ratio_17_18[i][17*60]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del C_1_ratio_17_18[i]
        
for i in date_key:
    k=0
    if C_1_ratio_18_19[i][18*60]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del C_1_ratio_18_19[i]
        
for i in C_1_ratio_8_9:
    for k in [480,500,520]:
        if C_1_ratio_8_9[i][k]==[] and C_1_ratio_8_9[i][k-20]!=[] and C_1_ratio_8_9[i][k+20]!=[]:
            C_1_ratio_8_9[i][k]=[np.mean([np.mean(C_1_ratio_8_9[i][k-20]),np.mean(C_1_ratio_8_9[i][k+20])])]
            
for i in C_1_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if C_1_ratio_9_10[i][k]==[] and C_1_ratio_9_10[i][k-20]!=[] and C_1_ratio_9_10[i][k+20]!=[]:
            C_1_ratio_9_10[i][k]=[np.mean([np.mean(C_1_ratio_9_10[i][k-20]),np.mean(C_1_ratio_9_10[i][k+20])])]
            
for i in C_1_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if C_1_ratio_17_18[i][k]==[] and C_1_ratio_17_18[i][k-20]!=[] and C_1_ratio_17_18[i][k+20]!=[]:
            C_1_ratio_17_18[i][k]=[np.mean([np.mean(C_1_ratio_17_18[i][k-20]),np.mean(C_1_ratio_17_18[i][k+20])])]
            
for i in C_1_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if C_1_ratio_18_19[i][k]==[] and C_1_ratio_18_19[i][k-20]!=[] and C_1_ratio_18_19[i][k+20]!=[]:
            C_1_ratio_18_19[i][k]=[np.mean([np.mean(C_1_ratio_18_19[i][k-20]),np.mean(C_1_ratio_18_19[i][k+20])])]
            
C_1_ratio_8_9_key=list(C_1_ratio_8_9.keys())
for i in C_1_ratio_8_9_key:
    k=0
    if C_1_ratio_8_9[i][480]==[]:k+=1
    if C_1_ratio_8_9[i][500]==[]:k+=1
    if C_1_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del C_1_ratio_8_9[i]
        
C_1_ratio_9_10_key=list(C_1_ratio_9_10.keys())
for i in C_1_ratio_9_10_key:
    k=0
    if C_1_ratio_9_10[i][540]==[]:k+=1
    if C_1_ratio_9_10[i][560]==[]:k+=1
    if C_1_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del C_1_ratio_9_10[i]

C_1_ratio_17_18_key=list(C_1_ratio_17_18.keys())
for i in C_1_ratio_17_18_key:
    k=0
    if C_1_ratio_17_18[i][17*60]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del C_1_ratio_17_18[i]

C_1_ratio_18_19_key=list(C_1_ratio_18_19.keys())
for i in  C_1_ratio_18_19_key:
    k=0
    if C_1_ratio_18_19[i][18*60]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del C_1_ratio_18_19[i]
        
for i in C_1_ratio_8_9:
    del C_1_ratio_8_9[i][8*60-20]
    del C_1_ratio_8_9[i][9*60]
    
for i in C_1_ratio_9_10:
    del C_1_ratio_9_10[i][9*60-20]
    del C_1_ratio_9_10[i][10*60]
    
for i in C_1_ratio_17_18:
    del C_1_ratio_17_18[i][17*60-20]
    del C_1_ratio_17_18[i][18*60]
    
for i in C_1_ratio_18_19:
    del C_1_ratio_18_19[i][18*60-20]
    del C_1_ratio_18_19[i][19*60]
    
for i in C_1_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in C_1_ratio_8_9[i]: 
        if C_1_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_8_9[i][j]))
            temp2.append(np.mean(C_1_ratio_8_9[i][j]))
        if C_1_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in C_1_ratio_9_10[i]: 
        if C_1_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_9_10[i][j]))
            temp2.append(np.mean(C_1_ratio_9_10[i][j]))
        if C_1_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in C_1_ratio_17_18[i]: 
        if C_1_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_17_18[i][j]))
            temp2.append(np.mean(C_1_ratio_17_18[i][j]))
        if C_1_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in C_1_ratio_18_19[i]: 
        if C_1_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_18_19[i][j]))
            temp2.append(np.mean(C_1_ratio_18_19[i][j]))
        if C_1_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
C_1_ratio_8_9_480=[]
C_1_ratio_8_9_500=[]
C_1_ratio_8_9_520=[]

for i in C_1_ratio_8_9:
    for j in C_1_ratio_8_9[i]:
        if j==480: C_1_ratio_8_9_480.extend(C_1_ratio_8_9[i][j])
        if j==500: C_1_ratio_8_9_500.extend(C_1_ratio_8_9[i][j])
        if j==520: C_1_ratio_8_9_520.extend(C_1_ratio_8_9[i][j])
    
    
C_1_ratio_9_10_540=[]
C_1_ratio_9_10_560=[]
C_1_ratio_9_10_580=[]

for i in C_1_ratio_9_10:
    for j in C_1_ratio_9_10[i]:
        if j==540: C_1_ratio_9_10_540.extend(C_1_ratio_9_10[i][j])
        if j==560: C_1_ratio_9_10_560.extend(C_1_ratio_9_10[i][j])
        if j==580: C_1_ratio_9_10_580.extend(C_1_ratio_9_10[i][j])
            
C_1_ratio_17_18_1020=[]
C_1_ratio_17_18_1040=[]
C_1_ratio_17_18_1060=[]

for i in C_1_ratio_17_18:
    for j in C_1_ratio_17_18[i]:
        if j==1020: C_1_ratio_17_18_1020.extend(C_1_ratio_17_18[i][j])
        if j==1040: C_1_ratio_17_18_1040.extend(C_1_ratio_17_18[i][j])
        if j==1060: C_1_ratio_17_18_1060.extend(C_1_ratio_17_18[i][j])
    
C_1_ratio_18_19_1080=[]
C_1_ratio_18_19_1100=[]
C_1_ratio_18_19_1120=[]

for i in C_1_ratio_18_19:
    for j in C_1_ratio_18_19[i]:
        if j==1080: C_1_ratio_18_19_1080.extend(C_1_ratio_18_19[i][j])
        if j==1100: C_1_ratio_18_19_1100.extend(C_1_ratio_18_19[i][j])
        if j==1120: C_1_ratio_18_19_1120.extend(C_1_ratio_18_19[i][j])
            
C_1_ratio_8_9_480.sort()
C_1_ratio_8_9_480_used=C_1_ratio_8_9_480[5:len(C_1_ratio_8_9_480)-5]

C_1_ratio_8_9_500.sort()
C_1_ratio_8_9_500_used=C_1_ratio_8_9_500[5:len(C_1_ratio_8_9_500)-5]

C_1_ratio_8_9_520.sort()
C_1_ratio_8_9_520_used=C_1_ratio_8_9_520[5:len(C_1_ratio_8_9_520)-5]

C_1_ratio_9_10_540.sort()
C_1_ratio_9_10_540_used=C_1_ratio_9_10_540[5:len(C_1_ratio_9_10_540)-5]

C_1_ratio_9_10_560.sort()
C_1_ratio_9_10_560_used=C_1_ratio_9_10_560[5:len(C_1_ratio_9_10_560)-5]

C_1_ratio_9_10_580.sort()
C_1_ratio_9_10_580_used=C_1_ratio_9_10_580[5:len(C_1_ratio_9_10_580)-5]

C_1_ratio_17_18_1020.sort()
C_1_ratio_17_18_1020_used=C_1_ratio_17_18_1020[5:len(C_1_ratio_17_18_1020)-5]

C_1_ratio_17_18_1040.sort()
C_1_ratio_17_18_1040_used=C_1_ratio_17_18_1040[5:len(C_1_ratio_17_18_1040)-5]

C_1_ratio_17_18_1060.sort()
C_1_ratio_17_18_1060_used=C_1_ratio_17_18_1060[5:len(C_1_ratio_17_18_1060)-5]

C_1_ratio_18_19_1080.sort()
C_1_ratio_18_19_1080_used=C_1_ratio_18_19_1080[5:len(C_1_ratio_18_19_1080)-5]

C_1_ratio_18_19_1100.sort()
C_1_ratio_18_19_1100_used=C_1_ratio_18_19_1100[5:len(C_1_ratio_18_19_1100)-5]

C_1_ratio_18_19_1120.sort()
C_1_ratio_18_19_1120_used=C_1_ratio_18_19_1120[5:len(C_1_ratio_18_19_1120)-5]

names=locals()
C_1_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    C_1_ratio.append(np.mean(names['C_1_ratio_'+str(i)+'_used']))
    
C_1_ratio_8=C_1_ratio[0:3]
C_1_ratio_9=C_1_ratio[3:6]
C_1_ratio_17=C_1_ratio[6:9]
C_1_ratio_18=C_1_ratio[9:12]



In [281]:
print(C_1_ratio_8)
print(C_1_ratio_9)
print(C_1_ratio_17)
print(C_1_ratio_18)

[1.0085446631871167, 1.0135826326873116, 0.9693400182600777]
[0.9649815049634551, 1.0068986583841009, 1.0131470826840883]
[0.9455038549039972, 0.9966929710379031, 1.0406337231620946]
[0.95167652707838, 1.0515667415053014, 0.9895062988612441]


In [282]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/C_1.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/C_1.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=8, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

#print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#print(test_compare)

train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/C_1.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/C_1.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in C_1_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in C_1_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in C_1_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in C_1_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0]) 

0.19453825303148206


In [283]:
real_value_df_C_1.loc[real_value_df_C_1['route'] == 'C_1']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_C_1_time=list(real_value_df_C_1['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_C_1_time:final_result3.append(i[0])
        

In [284]:
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    

#load the real value 
real_value_df_C_1=real_value_df.loc[real_value_df['route'] =='C_1']

real_value=list(real_value_df_C_1['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.2185103004349994

In [285]:
mape_list

[0.37682876572413254,
 0.016728671087792822,
 0.12136320157338426,
 0.4164941728695019,
 0.2240434980451946,
 0.5577886373260434,
 0.048723723364423065,
 0.14810047191826162,
 0.03705007142752906,
 0.22221634870872853,
 0.5057625759638333,
 0.13868785523963864,
 0.043955930206148075,
 0.2318794785948884,
 0.08415720364472377,
 0.281251519863485,
 0.1809352288914343,
 0.2953551503709842,
 0.08887894553172233,
 0.08778251339213086,
 0.3300326052144979,
 0.3324236347540944,
 0.08776125515742322,
 0.026899239204257738,
 0.12466345569278607,
 0.3701162316320727,
 0.08483175948713997,
 0.31042209849819,
 0.2061704639034163,
 0.3930502251086256,
 0.047391318607532325,
 0.3555362254421307,
 0.14768236089888806,
 0.1710410968697311,
 0.10470367654874503,
 0.09938458981280782,
 0.055929861356468574,
 0.0014540088765470163,
 0.3076365518547835,
 0.5835112688418501,
 0.5804291439398906,
 0.639279969412115,
 0.6037349975629114,
 0.42453661398748416,
 0.12213296918638135,
 0.0487834244581106,
 0.033

In [223]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[134.51, 197.94241618604457],
 [183.06, 198.9311952574531],
 [158.02, 187.61230200495416],
 [130.53, 195.76186094084278],
 [151.99, 196.97668346417055],
 [417.44, 192.7990887722987],
 [185.55, 203.2371371149768],
 [238.49, 212.1971608373272],
 [179.15, 178.11743860130912],
 [263.94, 196.81306534927597],
 [390.85, 185.19772466607287],
 [158.18, 191.66255811536826],
 [189.34, 192.61996749678642],
 [140.53, 184.21215674893227],
 [158.96, 170.58642710081486],
 [250.19, 177.99641102225328],
 [220.91, 179.1009880228095],
 [233.4, 161.76173101749887],
 [190.28, 170.51943199579594],
 [198.43, 178.03704505358908],
 [283.97, 196.53798579169725],
 [314.9, 217.1670766489008],
 [200.43, 180.32622464876644],
 [178.94, 181.2270057971143],
 [200.76, 173.3164948207908],
 [138.53, 201.88072087643408],
 [182.56, 210.65007563206592],
 [152.07, 211.95728866727436],
 [226.48, 205.22865770183458],
 [312.25, 216.33963682541682],
 [174.53, 212.43138006017577],
 [149.01, 234.72867909135184],
 [165.61, 220.8756

## C3


In [303]:

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
C_3_travel_times=travel_times['C_3']
C_3_travel_times_need={}
for i in C_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in C_3_travel_times_need:
            C_3_travel_times_need[i]=[]
        if i in C_3_travel_times_need:
            C_3_travel_times_need[i].extend(C_3_travel_times[i])
        
C_3_ratio_8_9={}
C_3_ratio_9_10={}
C_3_ratio_17_18={}
C_3_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    C_3_ratio_8_9[i]={}
    C_3_ratio_9_10[i]={}
    C_3_ratio_17_18[i]={}
    C_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        C_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        C_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        C_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        C_3_ratio_18_19[i][j]=[]

for i in C_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        C_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        C_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        C_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        C_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if C_3_ratio_8_9[i][480]==[]:k+=1
    if C_3_ratio_8_9[i][500]==[]:k+=1
    if C_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del C_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if C_3_ratio_9_10[i][540]==[]:k+=1
    if C_3_ratio_9_10[i][560]==[]:k+=1
    if C_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del C_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if C_3_ratio_17_18[i][17*60]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del C_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if C_3_ratio_18_19[i][18*60]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del C_3_ratio_18_19[i]
        
for i in C_3_ratio_8_9:
    for k in [480,500,520]:
        if C_3_ratio_8_9[i][k]==[] and C_3_ratio_8_9[i][k-20]!=[] and C_3_ratio_8_9[i][k+20]!=[]:
            C_3_ratio_8_9[i][k]=[np.mean([np.mean(C_3_ratio_8_9[i][k-20]),np.mean(C_3_ratio_8_9[i][k+20])])]
            
for i in C_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if C_3_ratio_9_10[i][k]==[] and C_3_ratio_9_10[i][k-20]!=[] and C_3_ratio_9_10[i][k+20]!=[]:
            C_3_ratio_9_10[i][k]=[np.mean([np.mean(C_3_ratio_9_10[i][k-20]),np.mean(C_3_ratio_9_10[i][k+20])])]
            
for i in C_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if C_3_ratio_17_18[i][k]==[] and C_3_ratio_17_18[i][k-20]!=[] and C_3_ratio_17_18[i][k+20]!=[]:
            C_3_ratio_17_18[i][k]=[np.mean([np.mean(C_3_ratio_17_18[i][k-20]),np.mean(C_3_ratio_17_18[i][k+20])])]
            
for i in C_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if C_3_ratio_18_19[i][k]==[] and C_3_ratio_18_19[i][k-20]!=[] and C_3_ratio_18_19[i][k+20]!=[]:
            C_3_ratio_18_19[i][k]=[np.mean([np.mean(C_3_ratio_18_19[i][k-20]),np.mean(C_3_ratio_18_19[i][k+20])])]
            
C_3_ratio_8_9_key=list(C_3_ratio_8_9.keys())
for i in C_3_ratio_8_9_key:
    k=0
    if C_3_ratio_8_9[i][480]==[]:k+=1
    if C_3_ratio_8_9[i][500]==[]:k+=1
    if C_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del C_3_ratio_8_9[i]
        
C_3_ratio_9_10_key=list(C_3_ratio_9_10.keys())
for i in C_3_ratio_9_10_key:
    k=0
    if C_3_ratio_9_10[i][540]==[]:k+=1
    if C_3_ratio_9_10[i][560]==[]:k+=1
    if C_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del C_3_ratio_9_10[i]

C_3_ratio_17_18_key=list(C_3_ratio_17_18.keys())
for i in C_3_ratio_17_18_key:
    k=0
    if C_3_ratio_17_18[i][17*60]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del C_3_ratio_17_18[i]

C_3_ratio_18_19_key=list(C_3_ratio_18_19.keys())
for i in  C_3_ratio_18_19_key:
    k=0
    if C_3_ratio_18_19[i][18*60]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del C_3_ratio_18_19[i]
        
for i in C_3_ratio_8_9:
    del C_3_ratio_8_9[i][8*60-20]
    del C_3_ratio_8_9[i][9*60]
    
for i in C_3_ratio_9_10:
    del C_3_ratio_9_10[i][9*60-20]
    del C_3_ratio_9_10[i][10*60]
    
for i in C_3_ratio_17_18:
    del C_3_ratio_17_18[i][17*60-20]
    del C_3_ratio_17_18[i][18*60]
    
for i in C_3_ratio_18_19:
    del C_3_ratio_18_19[i][18*60-20]
    del C_3_ratio_18_19[i][19*60]
    
for i in C_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in C_3_ratio_8_9[i]: 
        if C_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_8_9[i][j]))
            temp2.append(np.mean(C_3_ratio_8_9[i][j]))
        if C_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in C_3_ratio_9_10[i]: 
        if C_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_9_10[i][j]))
            temp2.append(np.mean(C_3_ratio_9_10[i][j]))
        if C_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in C_3_ratio_17_18[i]: 
        if C_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_17_18[i][j]))
            temp2.append(np.mean(C_3_ratio_17_18[i][j]))
        if C_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in C_3_ratio_18_19[i]: 
        if C_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_18_19[i][j]))
            temp2.append(np.mean(C_3_ratio_18_19[i][j]))
        if C_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
C_3_ratio_8_9_480=[]
C_3_ratio_8_9_500=[]
C_3_ratio_8_9_520=[]

for i in C_3_ratio_8_9:
    for j in C_3_ratio_8_9[i]:
        if j==480: C_3_ratio_8_9_480.extend(C_3_ratio_8_9[i][j])
        if j==500: C_3_ratio_8_9_500.extend(C_3_ratio_8_9[i][j])
        if j==520: C_3_ratio_8_9_520.extend(C_3_ratio_8_9[i][j])
    
    
C_3_ratio_9_10_540=[]
C_3_ratio_9_10_560=[]
C_3_ratio_9_10_580=[]

for i in C_3_ratio_9_10:
    for j in C_3_ratio_9_10[i]:
        if j==540: C_3_ratio_9_10_540.extend(C_3_ratio_9_10[i][j])
        if j==560: C_3_ratio_9_10_560.extend(C_3_ratio_9_10[i][j])
        if j==580: C_3_ratio_9_10_580.extend(C_3_ratio_9_10[i][j])
            
C_3_ratio_17_18_1020=[]
C_3_ratio_17_18_1040=[]
C_3_ratio_17_18_1060=[]

for i in C_3_ratio_17_18:
    for j in C_3_ratio_17_18[i]:
        if j==1020: C_3_ratio_17_18_1020.extend(C_3_ratio_17_18[i][j])
        if j==1040: C_3_ratio_17_18_1040.extend(C_3_ratio_17_18[i][j])
        if j==1060: C_3_ratio_17_18_1060.extend(C_3_ratio_17_18[i][j])
    
C_3_ratio_18_19_1080=[]
C_3_ratio_18_19_1100=[]
C_3_ratio_18_19_1120=[]

for i in C_3_ratio_18_19:
    for j in C_3_ratio_18_19[i]:
        if j==1080: C_3_ratio_18_19_1080.extend(C_3_ratio_18_19[i][j])
        if j==1100: C_3_ratio_18_19_1100.extend(C_3_ratio_18_19[i][j])
        if j==1120: C_3_ratio_18_19_1120.extend(C_3_ratio_18_19[i][j])
            
C_3_ratio_8_9_480.sort()
C_3_ratio_8_9_480_used=C_3_ratio_8_9_480[5:len(C_3_ratio_8_9_480)-5]

C_3_ratio_8_9_500.sort()
C_3_ratio_8_9_500_used=C_3_ratio_8_9_500[5:len(C_3_ratio_8_9_500)-5]

C_3_ratio_8_9_520.sort()
C_3_ratio_8_9_520_used=C_3_ratio_8_9_520[5:len(C_3_ratio_8_9_520)-5]

C_3_ratio_9_10_540.sort()
C_3_ratio_9_10_540_used=C_3_ratio_9_10_540[5:len(C_3_ratio_9_10_540)-5]

C_3_ratio_9_10_560.sort()
C_3_ratio_9_10_560_used=C_3_ratio_9_10_560[5:len(C_3_ratio_9_10_560)-5]

C_3_ratio_9_10_580.sort()
C_3_ratio_9_10_580_used=C_3_ratio_9_10_580[5:len(C_3_ratio_9_10_580)-5]

C_3_ratio_17_18_1020.sort()
C_3_ratio_17_18_1020_used=C_3_ratio_17_18_1020[5:len(C_3_ratio_17_18_1020)-5]

C_3_ratio_17_18_1040.sort()
C_3_ratio_17_18_1040_used=C_3_ratio_17_18_1040[5:len(C_3_ratio_17_18_1040)-5]

C_3_ratio_17_18_1060.sort()
C_3_ratio_17_18_1060_used=C_3_ratio_17_18_1060[5:len(C_3_ratio_17_18_1060)-5]

C_3_ratio_18_19_1080.sort()
C_3_ratio_18_19_1080_used=C_3_ratio_18_19_1080[5:len(C_3_ratio_18_19_1080)-5]

C_3_ratio_18_19_1100.sort()
C_3_ratio_18_19_1100_used=C_3_ratio_18_19_1100[5:len(C_3_ratio_18_19_1100)-5]

C_3_ratio_18_19_1120.sort()
C_3_ratio_18_19_1120_used=C_3_ratio_18_19_1120[5:len(C_3_ratio_18_19_1120)-5]

names=locals()
C_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    C_3_ratio.append(np.mean(names['C_3_ratio_'+str(i)+'_used']))
    
C_3_ratio_8=C_3_ratio[0:3]
C_3_ratio_9=C_3_ratio[3:6]
C_3_ratio_17=C_3_ratio[6:9]
C_3_ratio_18=C_3_ratio[9:12]



In [304]:
print(C_3_ratio_8)
print(C_3_ratio_9)
print(C_3_ratio_17)
print(C_3_ratio_18)

[0.9148328918283646, 1.0075582153926481, 1.0225619584493677]
[1.040282893182057, 0.9801969562756571, 0.9588049191875893]
[0.9750481330349163, 0.9561918653994332, 1.064546493530613]
[1.0017435848975271, 0.980500959518688, 0.9999905668560394]


In [305]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/C_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/C_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

#print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
#print(test_compare)

train_path = '/Users/vayne/Desktop/dm_pro_engin/data_feature/C_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/data_testing/C_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in C_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in C_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in C_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in C_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0]) 

0.16136689463949525


In [306]:
#有的时间窗口真实数据里是没有值的，所以要筛一下，去掉predicted value里的一些值
real_value_df_C_3.loc[real_value_df_C_3['route'] == 'C_3']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_C_3_time=list(real_value_df_C_3['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_C_3_time:final_result3.append(i[0])

pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    

#load the real value 
real_value_df_C_3=real_value_df.loc[real_value_df['route'] =='C_3']

real_value=list(real_value_df_C_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)



0.24126218559987836

In [307]:
for i in mape_list:
    if i>1:
        mape_list.remove(i)

np.mean(mape_list)

0.21263574279198705

In [308]:

test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[233.55, 191.5642774981399],
 [155.31, 194.41689797655874],
 [216.72, 183.7651254732655],
 [178.4, 173.15099367589593],
 [144.3, 169.37210775420888],
 [189.83, 163.82328594339432],
 [84.47, 175.5222981705879],
 [204.89, 175.21513997160838],
 [188.63, 181.9788691532709],
 [175.91, 200.42382196905274],
 [202.04, 203.40837162715613],
 [209.15, 209.2902540144762],
 [143.38, 197.20181049564442],
 [168.16, 192.8980341811518],
 [108.94, 176.2511500539585],
 [185.13, 172.8426630840541],
 [273.95, 192.33285265136354],
 [123.48, 188.25430920121073],
 [170.41, 191.99627653972001],
 [141.54, 190.89443366341493],
 [194.8, 193.73707936853452],
 [148.35, 183.30368743782486],
 [209.36, 172.71620794519413],
 [143.44, 189.12081974700564],
 [138.56, 185.4634487190835],
 [173.81, 206.47996616193763],
 [211.71, 193.7257218052302],
 [124.56, 176.7188875326968],
 [214.37, 179.35044245558197],
 [106.63, 183.55214792448436],
 [155.75, 172.9503175459331],
 [131.44, 169.17581122488386],
 [193.78, 171.0164080201

In [ ]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

In [272]:
summ=0.17570534823439676+0.133475149336903+0.2152616858735426+0.21764746772789725+0.2185103004349994+0.21263574279198705

In [273]:
summ/6

0.19553928239995433